In [3]:
import re
from collections import defaultdict, Counter
import math
import random
from scipy.optimize import minimize

class NgramLanguageModel:
    def __init__(self, n, lambdas=None):
        self.n = n
        self.model = defaultdict(Counter)
        self.vocab = set()
        if lambdas:
            self.lambdas = lambdas
        else:
            self.lambdas = [1.0 / n] * n 
        
    def preprocess(self, text):
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text).lower()
        tokens = text.split()
        return tokens
    
    def train(self, corpus):
        tokenized_lines = []
        word_counts = Counter()
        
        # Preprocess each line and update counts
        for line in corpus:
            tokens = self.preprocess(line)
            tokenized_lines.append(tokens)
            word_counts.update(tokens)
        
        # Handle OOV by converting words that appear < 3 times to <UNK>
        for word in word_counts:
            if word_counts[word] < 3:
                self.vocab.add('<UNK>')
            else:
                self.vocab.add(word)
                
        # Train n-gram model
        for tokens in tokenized_lines:
            tokens = ['<START>'] * (self.n - 1) + [token if token in self.vocab else '<UNK>' for token in tokens] + ['<STOP>']
            for i in range(len(tokens) - self.n + 1):
                context = tuple(tokens[i:i + self.n - 1])
                word = tokens[i + self.n - 1]
                self.model[context][word] += 1
        
    def get_ngram_prob(self, context, word):
        context_count = sum(self.model[context].values())
        word_count = self.model[context][word]
        vocab_size = len(self.vocab)
        return (word_count + 1) / (context_count + vocab_size) if context_count > 0 else 1 / vocab_size  # Add-one smoothing with vocab size

    def get_smoothed_prob(self, context, word):
        prob = 0.0
        for i in range(self.n):
            sub_context = context[len(context) - i:]
            prob += self.lambdas[i] * self.get_ngram_prob(sub_context, word)
        return prob

    def calculate_perplexity(self, corpus):
        total_log_prob = 0
        total_tokens = 0
        
        for line in corpus:
            tokens = ['<START>'] * (self.n - 1) + [token if token in self.vocab else '<UNK>' for token in self.preprocess(line)] + ['<STOP>']
            total_tokens += len(tokens) - (self.n - 1)
            
            for i in range(len(tokens) - self.n + 1):
                context = tuple(tokens[i:i + self.n - 1])
                word = tokens[i + self.n - 1]
                prob = self.get_smoothed_prob(context, word)
                if prob > 0:
                    total_log_prob += math.log(prob)
                else:
                    return float('inf')
                    
        avg_log_prob = total_log_prob / total_tokens
        perplexity = math.exp(-avg_log_prob)
        return perplexity

# Read data files
def load_corpus(file_path):
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        return file.readlines()

def find_best_lambdas(model, train_corpus, dev_corpus):
    def objective_function(lambdas):
        model.lambdas = lambdas
        return model.calculate_perplexity(dev_corpus)
    
    # Initial lambda values for optimization
    initial_lambdas = [1.0 / model.n] * model.n

    # Constraints: lambdas must sum to 1 and be non-negative
    constraints = ({'type': 'eq', 'fun': lambda lambdas: sum(lambdas) - 1})
    bounds = [(0, 1) for _ in range(model.n)]

    result = minimize(objective_function, initial_lambdas, bounds=bounds, constraints=constraints, method='SLSQP')
    return result.x

if __name__ == "__main__":
    train_file = '1b_benchmark.train.tokens'
    dev_file = '1b_benchmark.dev.tokens'
    test_file = '1b_benchmark.test.tokens'

    # Load data
    train_corpus = load_corpus(train_file)
    dev_corpus = load_corpus(dev_file)
    test_corpus = load_corpus(test_file)

    # Train and evaluate models for unigram, bigram, and trigram
    for n in [1, 2, 3]:
        print(f"Training {n}-gram model...")
        model = NgramLanguageModel(n)
        model.train(train_corpus)

        # Find the best lambda values for linear interpolation
        best_lambdas = find_best_lambdas(model, train_corpus, dev_corpus)
        model.lambdas = best_lambdas
        print(f"Best lambdas for {n}-gram: {best_lambdas}")

        train_perplexity = model.calculate_perplexity(train_corpus)
        dev_perplexity = model.calculate_perplexity(dev_corpus)
        test_perplexity = model.calculate_perplexity(test_corpus)

        print(f"{n}-gram Training Perplexity: {train_perplexity}")
        print(f"{n}-gram Development Perplexity: {dev_perplexity}")
        print(f"{n}-gram Test Perplexity: {test_perplexity}")

Training 1-gram model...
Best lambdas for 1-gram: [1.]
1-gram Training Perplexity: 1130.0970935401174
1-gram Development Perplexity: 1040.7830165470816
1-gram Test Perplexity: 1040.1657826290268
Training 2-gram model...
Best lambdas for 2-gram: [3.46725051e-14 1.00000000e+00]
2-gram Training Perplexity: 2013.9531441572253
2-gram Development Perplexity: 2429.310848371299
2-gram Test Perplexity: 2414.680451657738
Training 3-gram model...
Best lambdas for 3-gram: [0. 0. 1.]
3-gram Training Perplexity: 6750.764084596879
3-gram Development Perplexity: 11054.39056358828
3-gram Test Perplexity: 11004.951021273084
